`CIS_bdpm.txt`
- Forme
- Route d'administration
- Code CIS
- Nom du medicament

`CIS_COMPO_bdpm.txt`
- Code CIS
- Ingredient Actif
- Substance Active vs Fraction Therapeutique


In [1]:
import pandas as pd

In [2]:
columns_cis_bdpm = '''code_cis
denomination
form
roa
status_amm
type_amm
commercialisation_state
date_amm
statutbdm
num_authorization_eu
titulaire
surveillance'''.split('\n')
df_bdpm = pd.read_csv("CIS_bdpm.txt",names=columns_cis_bdpm,delimiter='\t',index_col=False)
df_bdpm

,code_cis,denomination,form,roa,status_amm,type_amm,commercialisation_state,date_amm,statutbdm,num_authorization_eu,titulaire,surveillance
0,61266250,"A 313 200 000 UI POUR CENT, pommade",pommade,cutanée,Autorisation active,Procédure nationale,Commercialisée,12/03/1998,NaN,NaN,PHARMA DEVELOPPEMENT,Non
1,62869109,"A 313 50 000 U.I., capsule molle",capsule molle,orale,Autorisation active,Procédure nationale,Commercialisée,07/07/1997,NaN,NaN,PHARMA DEVELOPPEMENT,Non
2,61876780,"ABACAVIR ARROW 300 mg, comprimé pelliculé sécable",comprimé pelliculé sécable,orale,Autorisation active,Procédure décentralisée,Commercialisée,22/10/2019,NaN,NaN,ARROW GENERIQUES,Non
3,62401060,"ABACAVIR MYLAN 300 mg, comprimé pelliculé sécable",comprimé pelliculé sécable,orale,Autorisation active,Procédure décentralisée,Commercialisée,21/02/2018,NaN,NaN,MYLAN SAS,Non
4,63797011,"ABACAVIR SANDOZ 300 mg, comprimé pelliculé séc...",comprimé pelliculé sécable,orale,Autorisation active,Procédure décentralisée,Commercialisée,30/12/2016,NaN,NaN,SANDOZ,Non
...,...,...,...,...,...,...,...,...,...,...,...,...
15690,66425614,"ZYTIGA 250 mg, comprimé",comprimé,orale,Autorisation active,Procédure centralisée,Non commercialisée,05/09/2011,Warning disponibilité,EU/1/11/714,JANSSEN CILAG INTERNATIONAL NV,Oui
15691,67337081,"ZYTIGA 500 mg, comprimé pelliculé",comprimé pelliculé,orale,Autorisation active,Procédure centralisée,Commercialisée,09/11/2016,NaN,EU/1/11/714,JANSSEN CILAG INTERNATIONAL NV,Non
15692,63095061,"ZYVOXID 100 mg/5 ml, granulés pour suspension ...",granulés pour suspension buvable,orale,Autorisation active,Procédure de reconnaissance mutuelle,Commercialisée,31/08/2001,NaN,NaN,PFIZER HOLDING FRANCE,Non
15693,63283095,"ZYVOXID 2 mg/ml, solution pour perfusion",solution pour perfusion,intraveineuse,Autorisation active,Procédure de reconnaissance mutuelle,Commercialisée,31/08/2001,NaN,NaN,PFIZER HOLDING FRANCE,Non


In [3]:
columns_cis_compo = '''code_cis
designation
substance_code
denomination
dose
dose_ref
composant_type
num_linked'''.split('\n')
print(columns_cis_compo)
df_compo = pd.read_csv("CIS_COMPO_bdpm.txt",names=columns_cis_compo,delimiter='\t',index_col=False)

['code_cis', 'designation', 'substance_code', 'denomination', 'dose', 'dose_ref', 'composant_type', 'num_linked']


/home/james/programs/miniconda3/envs/ml-env/lib/python3.7/site-packages/pandas/util/_decorators.py:311: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return func(*args, **kwargs)


In [4]:
units_conversion = {
    'mg' : 10 ** -3,
    'microgrammes' : 10 ** -6,
}


def gather_ocrx_medical_term(medical_term):
    df = pd.read_csv(f"/home/james/Datasets/ocrx-owls/match-ocrx/match_{medical_term}_fr.csv")
    labels = df['sLabel'].values
    codes = df['s'].values
    return [(str(val),str(val_code)) for val, val_code in zip(labels,codes)]

def parse_dosage(dose):
    splitted = dose.split(" ")
    unit = splitted[-1]
    if unit in units_conversion:
        pass
# print('\n'.join([str(el) for el in df_compo['dose'].unique()]))
ocrx_terms = ['form','roa','substance']
ocrx_dfs = [gather_ocrx_medical_term(term) for term in ocrx_terms]

In [5]:
def convert_to_upper(df,cols_name):
    for col_name in cols_name:
        df[col_name] = df[col_name].str.upper() 
# df_bdpm['form'] = df_bdpm['form'].str.upper()


Mots qu'on peut couper durant le matching:

BASE
MONOHYDRATE
DEHYDRATE
HYDRATE

In [6]:
sum(["HOMÉOPATHIQUE" in val for val in df_compo.denomination.values ])

8022

In [7]:
df_compo.shape

(32343, 8)

In [8]:
def process_term(el):
    if el.endswith("BASE"):
        return ' '.join(el.split(" ")[:-1]).strip()
    if el.endswith(")"):
        return (el.split("(")[-1][:-1] + " " + ' '.join(el.split("(")[0].split(' '))).strip()
    return el.strip()
df_compo['denomination'] = df_compo['denomination'].apply(process_term)

In [9]:
process_term("RANITIDINE (CHLORHYDRATE DE)")

'CHLORHYDRATE DE RANITIDINE'

In [10]:
process_term("RANITIDINE BASE")

'RANITIDINE'

In [11]:
upper_cols = ['form','roa']
convert_to_upper(df_bdpm,upper_cols)

In [12]:
df_bdpm['roa'] = df_bdpm['roa'].str.split(';')

In [13]:
df_bdpm['roa'].iloc[107]

['CUTANÉE', 'ORALE', 'SUBLINGUALE']

In [14]:
df_compo.loc[df_compo.dose == '45 - 70 mg']

,code_cis,designation,substance_code,denomination,dose,dose_ref,composant_type,num_linked
222,60074795,solution,15614,PROTÉINES PLASMATIQUES HUMAINES,45 - 70 mg,1 ml de solution,SA,1


In [15]:
df_bdpm[['code_cis','roa','form']]

,code_cis,roa,form
0,61266250,[CUTANÉE],POMMADE
1,62869109,[ORALE],CAPSULE MOLLE
2,61876780,[ORALE],COMPRIMÉ PELLICULÉ SÉCABLE
3,62401060,[ORALE],COMPRIMÉ PELLICULÉ SÉCABLE
4,63797011,[ORALE],COMPRIMÉ PELLICULÉ SÉCABLE
...,...,...,...
15690,66425614,[ORALE],COMPRIMÉ
15691,67337081,[ORALE],COMPRIMÉ PELLICULÉ
15692,63095061,[ORALE],GRANULÉS POUR SUSPENSION BUVABLE
15693,63283095,[INTRAVEINEUSE],SOLUTION POUR PERFUSION


In [23]:
def construct_vocab(df,column,is_list=False):
    vocab_set = None
    is_list = type(df[column].values[0]) == list
    if is_list:
        vocab_set = {el for els in df[column].values for el in els}
    else:
        vocab_set = {el for el in df[column].values}
    df = pd.DataFrame()
   
    df['ansm_code'] = [str(i) for i in range(len(vocab_set))]
    df['ansm_label'] = list(vocab_set)
    df = df.dropna().drop_duplicates().sort_values('ansm_label')
    return df

In [26]:
import os
if not os.path.exists("export"):
    os.mkdir("export")
bdpm_roa_df = construct_vocab(df_bdpm,"roa")
bdpm_form_df = construct_vocab(df_bdpm,"form")
bdpm_substance_df = construct_vocab(df_compo,'denomination')

bdpm_dfs = [bdpm_form_df,bdpm_roa_df,bdpm_substance_df]
names = ['form','roa','active_ingredient']
for name, df in zip(names,bdpm_dfs):
    df.to_csv(f"export/ansm_{name}_fr.csv")

In [27]:
def export_drug(old_df):
    df = old_df[['code_cis','denomination']].copy()
    df.columns = ['ansm_code','ansm_label']
    df = df.drop_duplicates().dropna().sort_values('ansm_label')
    df.to_csv("export/ansm_drug_fr.csv")

export_drug(df_bdpm)

In [28]:
!nemo .

In [18]:
def convert_ocrx_to_df(ocrx_elems):
    return [pd.DataFrame(ocrx_df,columns=['ocrx_label','ocrx_code']) for ocrx_df in ocrx_elems]

ocrx_dfs = convert_ocrx_to_df(ocrx_dfs)
ocrx_form_df, ocrx_roa_df, ocrx_substance_df = ocrx_dfs

In [19]:
from pytrie import Trie
def make_trie(ocrx_def_df):
    return {row['ocrx_label']: row['ocrx_code'] for _, row in ocrx_def_df.iterrows()}

def make_trie_ansm(ansm_def_df):
    return {row['ansm_label']: row['ansm_code'] for _, row in ansm_def_df.iterrows()}

def get_prefix(trie,term):
    words = term.split(' ')
    for i in reversed(range(1,len(words)+1)):
        term = ' '.join(words[:i])
        if term in trie:
            return (term, trie[term])
    return None
        
bdpm_dfs = [bdpm_form_df,bdpm_roa_df,bdpm_substance_df]
ocrx_form_trie,ocrx_roa_trie,ocrx_substance_trie = [make_trie(el) for el in ocrx_dfs]
ansm_tries = [make_trie_ansm(el) for el in bdpm_dfs]

In [57]:
import os
DIR = "match-ocrx-full"
def print_stats(df_merged,type_fn,name='ansm'):
    # total_match
    print("TOTAL MATCH:")
    total_df = df_merged.dropna()
    print(total_df.head())
    print(total_df.shape)
    total_df.to_csv(f"{DIR}/match_{type_fn}_total_match.csv")
    # left missing
    print("ONLY OCRX")
    only_ocrx = df_merged.loc[df_merged['ansm_code'].isna()]
    print(only_ocrx.head())
    print(only_ocrx.shape)
    only_ocrx.to_csv(f"{DIR}/match_{type_fn}_only_ocrx.csv")
    # right missing
    print(f"ONLY {name.upper()}")
    only_ansm = df_merged.loc[df_merged['ocrx_code'].isna()]
    print(only_ansm.head())
    print(only_ansm.shape)
    only_ansm.to_csv(f"{DIR}/match_{type_fn}_only_{name}.csv")
    stats = {'total' : total_df.shape[0], 'ocrx' : only_ocrx.shape[0], name : only_ansm.shape[0]}
    return stats

def add_prefixes(df,trie,trie_ansm):
    def process_row(row,trie):
        if type(row['ocrx_code']) == str and type(row['ansm_code']) == str:
            return (row['ocrx_code'], row['ocrx_label'], 'EXACT')
        elif str(row['ocrx_code']) == 'nan':
            result = get_prefix(trie,row['ansm_label'])
            if result is None:
                return (None, None, 'NONE')
            else:
                prefix_ocrx, ocrx_code = result
                return (ocrx_code, prefix_ocrx, 'PREFIX')
        else:
            return (row['ocrx_code'], row['ocrx_label'], 'NA')
    def process_row_ansm(row,trie):
        if type(row['ocrx_code']) == str and type(row['ansm_code']) == str:
            return (row['ansm_code'], row['ansm_label'], 'EXACT')
        elif str(row['ansm_code']) == 'nan':
            result = get_prefix(trie,row['ocrx_label'])
            if result is None:
                return (None, None, 'NONE')
            else:
                prefix_ansm, ansm_code = result
                return (ansm_code, prefix_ansm, 'PREFIX')
        else:
            return (row['ansm_code'], row['ansm_label'], 'NA')
        
            
#     import ipdb; ipdb.set_trace()
    els = ['ocrx','ansm']
    new_df = df.copy()
    for el, trie_el, process_fn in zip(els,[trie,trie_ansm],[process_row,process_row_ansm]):
        results = [process_fn(row,trie_el) for _, row in df.iterrows()]
        codes = [result[0] for result in results]
        full_terms = [result[1] for result in results]
        match_types = [result[2] for result in results]
        new_df[f'{el}_code'] = codes
        new_df[f'{el}_label'] = full_terms
        new_df[f'{el}_match_type'] = match_types
    return new_df

filenames = ['form','roa','substance']
if not os.path.exists(DIR):
    os.mkdir(DIR)
ocrxs = [ocrx_form_df,ocrx_roa_df,ocrx_substance_df]
ansms = [bdpm_form_df,bdpm_roa_df,bdpm_substance_df]
tries = [ocrx_form_trie,ocrx_roa_trie,ocrx_substance_trie]

all_stats = dict()
for ocrx, ansm, filename, trie, trie_ansm in zip(ocrxs,ansms,filenames,tries,ansm_tries):
    fullpath = f"{DIR}/match_{filename}.csv"
    df_merged = pd.merge(ocrx,ansm,how="outer",left_on=['ocrx_label'], right_on=['ansm_label'])
#     import ipdb; ipdb.set_trace()
    df_merged_with_prefix = add_prefixes(df_merged,trie,trie_ansm)
    all_stats[filename] = print_stats(df_merged_with_prefix,filename)
    df_merged_with_prefix.to_csv(fullpath)

TOTAL MATCH:
                                            ocrx_label  \
116                                            CAPSULE   
117                                            CAPSULE   
118  CAPSULE (A LIBÉRATION PROLONGÉE SUR 12HR) PAR ...   
119  CAPSULE (A LIBÉRATION PROLONGÉE SUR 12HR) PAR ...   
120  CAPSULE (A LIBÉRATION PROLONGÉE SUR 16HR) PAR ...   

                              ocrx_code ansm_label ansm_code ocrx_match_type  \
116  http://www.ocrx.ca/OCRx/4000000017    CAPSULE       319           EXACT   
117  http://www.ocrx.ca/OCRx/4000000017    CAPSULE       319           EXACT   
118  http://www.ocrx.ca/OCRx/4002221882    CAPSULE       319              NA   
119  http://www.ocrx.ca/OCRx/4002221882    CAPSULE       319              NA   
120  http://www.ocrx.ca/OCRx/4002222882    CAPSULE       319              NA   

    ansm_match_type  
116           EXACT  
117           EXACT  
118          PREFIX  
119          PREFIX  
120          PREFIX  
(1571, 6)
ONLY OCRX
     

In [58]:
all_stats

{'form': {'total': 1571, 'ocrx': 518, 'ansm': 74},
 'roa': {'total': 27, 'ocrx': 72, 'ansm': 41},
 'substance': {'total': 212029, 'ocrx': 676477, 'ansm': 1753}}

# Indications thérapeutiques

In [21]:
import pandas as pd

patholo_csv = "cis-pathologie.csv"
df_patho = pd.read_csv(patholo_csv,sep='\t')
df_patho.columns = ['code_cis', 'indications']
final_df_indications = pd.merge(pd.merge(df_bdpm,df_compo,on="code_cis",suffixes=['_bdpm','_compo']),df_patho,on="code_cis")
final_df_indications.to_csv("df_indications.csv")

In [22]:
final_df_indications.shape[0]

23885

In [23]:
df_patho.shape[0]

15338

In [24]:
df_compo.shape[0]

32343

In [25]:
df_bdpm.shape[0]

15695

In [26]:
els = [df for i, df in final_df_indications.groupby('code_cis') if df.shape[0] > 1]

In [27]:
els[0]

,code_cis,denomination_bdpm,form,roa,status_amm,type_amm,commercialisation_state,date_amm,statutbdm,num_authorization_eu,titulaire,surveillance,designation,substance_code,denomination_compo,dose,dose_ref,composant_type,num_linked,indications
18915,60002504,"RANITIDINE BIOGARAN 150 mg, comprimé effervescent",COMPRIMÉ EFFERVESCENT(E),[ORALE],Autorisation abrogée,Procédure nationale,Non commercialisée,04/07/1989,Warning disponibilité,NaN,SAINT GERMAIN,Non,comprimé,25783,CHLORHYDRATE DE RANITIDINE,168 mg,un comprimé,SA,1,Reflux gastro-oesophagien
18916,60002504,"RANITIDINE BIOGARAN 150 mg, comprimé effervescent",COMPRIMÉ EFFERVESCENT(E),[ORALE],Autorisation abrogée,Procédure nationale,Non commercialisée,04/07/1989,Warning disponibilité,NaN,SAINT GERMAIN,Non,comprimé,25783,CHLORHYDRATE DE RANITIDINE,168 mg,un comprimé,SA,1,Ulcère duodénal
18917,60002504,"RANITIDINE BIOGARAN 150 mg, comprimé effervescent",COMPRIMÉ EFFERVESCENT(E),[ORALE],Autorisation abrogée,Procédure nationale,Non commercialisée,04/07/1989,Warning disponibilité,NaN,SAINT GERMAIN,Non,comprimé,25783,CHLORHYDRATE DE RANITIDINE,168 mg,un comprimé,SA,1,Ulcère gastrique
18918,60002504,"RANITIDINE BIOGARAN 150 mg, comprimé effervescent",COMPRIMÉ EFFERVESCENT(E),[ORALE],Autorisation abrogée,Procédure nationale,Non commercialisée,04/07/1989,Warning disponibilité,NaN,SAINT GERMAIN,Non,comprimé,49632,RANITIDINE,150 mg,un comprimé,FT,1,Reflux gastro-oesophagien
18919,60002504,"RANITIDINE BIOGARAN 150 mg, comprimé effervescent",COMPRIMÉ EFFERVESCENT(E),[ORALE],Autorisation abrogée,Procédure nationale,Non commercialisée,04/07/1989,Warning disponibilité,NaN,SAINT GERMAIN,Non,comprimé,49632,RANITIDINE,150 mg,un comprimé,FT,1,Ulcère duodénal
18920,60002504,"RANITIDINE BIOGARAN 150 mg, comprimé effervescent",COMPRIMÉ EFFERVESCENT(E),[ORALE],Autorisation abrogée,Procédure nationale,Non commercialisée,04/07/1989,Warning disponibilité,NaN,SAINT GERMAIN,Non,comprimé,49632,RANITIDINE,150 mg,un comprimé,FT,1,Ulcère gastrique


In [28]:
import pandas as pd
import ast
final_df_indications = pd.read_csv("df_indications.csv").astype(str)
def parse_ast(df):
    cols = ['roa']
    for col in cols:
        df[col] = [ast.literal_eval(val) for val in df[col].values]
#         (ast.literal_eval)
parse_ast(final_df_indications)
synonym_df = pd.read_csv("bdpm-pathologies.csv",sep=';').dropna()
synonym_df.columns = ['indications','synonyms']
synonym_df.synonyms = synonym_df['synonyms'].astype('str').apply(lambda el: el.split(':')[-1] if el is not None else el)

In [29]:
synonym_dict = {row['indications'] : row['synonyms'] for _, row in synonym_df.iterrows()}

In [30]:
def convert_to_list_indication(el):
    if el in synonym_dict:
        return [el,synonym_dict[el]]
    else:
        return [el]
final_df_indications['indications'] = final_df_indications['indications'].apply(convert_to_list_indication)

In [31]:
final_df_indications['roa'].iloc[0]

['CUTANÉE']

In [32]:
if 'denomination_compo' in final_df_indications.columns:
    final_df_indications = final_df_indications.rename(columns={'denomination_compo':'substance'})

In [33]:
def convert_cols_to_list(df,cols):
    for col in cols:
        df[col] = df[col].apply(lambda el: [el] if type(el) != list else el)

def load_ocrx_matches(els):
    return {el: pd.read_csv(f'match-ocrx/match_{el}.csv',dtype=str) for el in els}

convert_cols_to_list(final_df_indications,['form','roa','substance'])
matches = load_ocrx_matches(['form','roa','substance'])

def create_extend_to_ocrx(el_type):
    def extend(el):
        df = matches[el_type]
        cols = ['ansm_label','ocrx_label','ansm_code','ocrx_code', 'ansm_match_type', 'ocrx_match_type']
#         import ipdb; ipdb.set_trace()
        return [dict(df.loc[(df['ansm_label'] == el_i)][cols].iloc[0]) for el_i in el]
    return extend

def extend_all_df_to_ocrx(df):
    new_df = df.copy()
    cols = ['form','roa','substance']
#     import ipdb; ipdb.set_trace()
    for col in cols:
        extender = create_extend_to_ocrx(col)
        new_df[col] = df[col].apply(extender)
    return new_df
df_with_ocrx = extend_all_df_to_ocrx(final_df_indications)

In [34]:
dict(matches['form'].iloc[0])

{'Unnamed: 0': '0',
 'ocrx_label': 'ANNEAU',
 'ocrx_code': 'http://www.ocrx.ca/OCRx/4000000058',
 'ansm_label': nan,
 'ansm_code': nan,
 'ocrx_match_type': nan,
 'ansm_match_type': 'NONE'}

In [35]:
df_with_ocrx['form'][0]

[{'ansm_label': 'POMMADE',
  'ocrx_label': 'POMMADE',
  'ansm_code': '298',
  'ocrx_code': 'http://www.ocrx.ca/OCRx/4000000083',
  'ansm_match_type': 'EXACT',
  'ocrx_match_type': 'EXACT'}]

In [36]:
df_with_ocrx.to_csv("df_ocrx.csv")

In [37]:
df_with_ocrx

,Unnamed: 0,code_cis,denomination_bdpm,form,roa,status_amm,type_amm,commercialisation_state,date_amm,statutbdm,...,titulaire,surveillance,designation,substance_code,substance,dose,dose_ref,composant_type,num_linked,indications
0,0,61266250,"A 313 200 000 UI POUR CENT, pommade","[{'ansm_label': 'POMMADE', 'ocrx_label': 'POMM...","[{'ansm_label': 'CUTANÉE', 'ocrx_label': nan, ...",Autorisation active,Procédure nationale,Commercialisée,12/03/1998,nan,...,PHARMA DEVELOPPEMENT,Non,pommade,77887,[{'ansm_label': 'CONCENTRAT DE VITAMINE A SYNT...,200 000 UI,100 g de pommade,SA,1.0,"[Dermatite de contact, Eczéma de contact]"
1,1,61266250,"A 313 200 000 UI POUR CENT, pommade","[{'ansm_label': 'POMMADE', 'ocrx_label': 'POMM...","[{'ansm_label': 'CUTANÉE', 'ocrx_label': nan, ...",Autorisation active,Procédure nationale,Commercialisée,12/03/1998,nan,...,PHARMA DEVELOPPEMENT,Non,pommade,77887,[{'ansm_label': 'CONCENTRAT DE VITAMINE A SYNT...,200 000 UI,100 g de pommade,SA,1.0,"[Eczéma de contact, Dermatite de contact]"
2,2,66513085,"ABASAGLAR 100 unités/ml, solution injectable e...","[{'ansm_label': 'SOLUTION INJECTABLE', 'ocrx_l...","[{'ansm_label': 'SOUS-CUTANÉE', 'ocrx_label': ...",Autorisation active,Procédure centralisée,Commercialisée,09/09/2014,nan,...,ELI LILLY NEDERLAND BV (PAY-BAS),Non,solution,44461,"[{'ansm_label': 'INSULINE GLARGINE', 'ocrx_lab...",100 unités,1 ml,SA,1.0,[Diabète]
3,3,64332894,"ABASAGLAR 100 unités/ml, solution injectable e...","[{'ansm_label': 'SOLUTION INJECTABLE', 'ocrx_l...","[{'ansm_label': 'SOUS-CUTANÉE', 'ocrx_label': ...",Autorisation active,Procédure centralisée,Commercialisée,09/09/2014,nan,...,ELI LILLY NEDERLAND BV (PAY-BAS),Non,solution,44461,"[{'ansm_label': 'INSULINE GLARGINE', 'ocrx_lab...",100 unités,1 ml,SA,1.0,[Diabète]
4,4,66207341,"ABELCET 5 mg/ml, suspension à diluer pour perf...",[{'ansm_label': 'SUSPENSION À DILUER POUR PERF...,"[{'ansm_label': 'INTRAVEINEUSE', 'ocrx_label':...",Autorisation active,Procédure nationale,Commercialisée,10/06/1997,nan,...,ACINO FRANCE,Non,suspension,1150,"[{'ansm_label': 'AMPHOTÉRICINE B', 'ocrx_label...",5 mg,1 mL de suspension,SA,1.0,[Aspergillose]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23880,23880,68664125,"ZYPREXA VELOTAB 5 mg, comprimé orodispersible","[{'ansm_label': 'COMPRIMÉ ORODISPERSIBLE', 'oc...","[{'ansm_label': 'ORALE', 'ocrx_label': 'ORALE'...",Autorisation active,Procédure centralisée,Commercialisée,03/02/2000,nan,...,ELI LILLY (PAYS BAS),Non,comprimé,17723,"[{'ansm_label': 'OLANZAPINE', 'ocrx_label': 'O...",5 mg,un comprimé,SA,1.0,[Schizophrénie]
23881,23881,65016707,"ZYRTEC 10 mg/ml, solution buvable en gouttes","[{'ansm_label': 'SOLUTION BUVABLE EN GOUTTES',...","[{'ansm_label': 'ORALE', 'ocrx_label': 'ORALE'...",Autorisation active,Procédure de reconnaissance mutuelle,Commercialisée,14/11/1996,nan,...,UCB PHARMA,Non,solution,60929,[{'ansm_label': 'DICHLORHYDRATE DE CÉTIRIZINE'...,"1,000 g",100 ml,SA,1.0,[Rhinite allergique]
23882,23882,65016707,"ZYRTEC 10 mg/ml, solution buvable en gouttes","[{'ansm_label': 'SOLUTION BUVABLE EN GOUTTES',...","[{'ansm_label': 'ORALE', 'ocrx_label': 'ORALE'...",Autorisation active,Procédure de reconnaissance mutuelle,Commercialisée,14/11/1996,nan,...,UCB PHARMA,Non,solution,60929,[{'ansm_label': 'DICHLORHYDRATE DE CÉTIRIZINE'...,"1,000 g",100 ml,SA,1.0,[Urticaire]
23883,23883,64949486,"ZYRTECSET 10 mg, comprimé pelliculé sécable","[{'ansm_label': 'COMPRIMÉ PELLICULÉ SÉCABLE', ...","[{'ansm_label': 'ORALE', 'ocrx_label': 'ORALE'...",Autorisation active,Procédure nationale,Commercialisée,28/10/1994,nan,...,UCB PHARMA,Non,comprimé,60929,[{'ansm_label': 'DICHLORHYDRATE DE CÉTIRIZINE'...,10 mg,un comprimé,SA,1.0,[Rhinite allergique]


In [38]:
from pytrie import Trie


# Lire le fichier HTML

In [39]:
import pandas as pd
df_rcp = pd.read_csv("CIS_RCP.csv",sep='\t')

In [40]:
df_rcp.columns

Index(['Code_CIS', 'RCP_html'], dtype='object')

In [41]:
import os
if not os.path.exists("htmls"):
    os.mkdir("htmls")
    for _, row in df_rcp.iterrows():
        content = row['RCP_html']
        if type(content) != str:
            continue
        code_cis = row['Code_CIS']
        with open(f"htmls/{code_cis}.html","w") as f:
            f.write(content)

In [42]:
from lxml import etree as et
import re
def strip_html(text):
#     return text
    if text is None:
        return None
    return re.sub('<[^<]+?>', '', text)
def extract_ade(content):
    if type(content) != str:
        return None
    lines = content.split('\n')
#     import ipdb; ipdb.set_trace()
    start_indices = [i for i,line in enumerate(lines) if 'RcpEffetsIndesirables' in line]
    if len(start_indices) == 0:
        return None
    start_index = start_indices[0]
    end_index = [start_index+j for j, line in enumerate(lines[start_index:]) if 'AmmAnnexeTitre2' in line and '"Rcp' in line]
    if len(end_index) <= 1:
        return ('\n'.join(lines[start_index:]))
    elif len(end_index) > 1:
        return ('\n'.join(lines[start_index:end_index[1]]))

In [43]:
df_rcp['adverse_events_html'] = df_rcp['RCP_html'].apply(extract_ade)
df_rcp['adverse_events_text'] = df_rcp['adverse_events_html'].apply(strip_html)

In [44]:
df_rcp.columns = ['code_cis', 'rcp_html', 'adverse_events_html', 'adverse_events_text']

In [45]:
df_rcp

,code_cis,rcp_html,adverse_events_html,adverse_events_text
0,61266250,"<!DOCTYPE html>\n<html>\n<body><div id=""textDo...","<p class=""AmmAnnexeTitre2""><a name=""RcpEffetsI...",4.8. Effets indésirables \nPossibilité de réa...
1,62869109,"<!DOCTYPE html>\n<html>\n<body><div id=""textDo...","<p class=""AmmAnnexeTitre2""><a name=""RcpEffetsI...",4.8. Effets indésirables \nTout effet indésir...
2,61876780,"<!DOCTYPE html>\n<html>\n<body><div id=""textDo...","<p class=""AmmAnnexeTitre2""><a name=""RcpEffetsI...",4.8. Effets indésirables \nPour la plupart de...
3,62401060,"<!DOCTYPE html>\n<html>\n<body><div id=""textDo...","<p class=""AmmAnnexeTitre2""><a name=""RcpEffetsI...",4.8. Effets indésirables \nPour la plupart de...
4,63797011,"<!DOCTYPE html>\n<html>\n<body><div id=""textDo...","<p class=""AmmAnnexeTitre2""><a name=""RcpEffetsI...",4.8. Effets indésirables \nPour la plupart de...
...,...,...,...,...
15644,66425614,NaN,None,None
15645,67337081,NaN,None,None
15646,63095061,"<!DOCTYPE html>\n<html>\n<body><div id=""textDo...","<p class=""AmmAnnexeTitre2""><a name=""RcpEffetsI...",4.8. Effets indésirables \nLe tableau ci-dess...
15647,63283095,"<!DOCTYPE html>\n<html>\n<body><div id=""textDo...","<p class=""AmmAnnexeTitre2""><a name=""RcpEffetsI...",4.8. Effets indésirables \nLe tableau ci-dess...


In [46]:
df_ocrx = pd.read_csv("df_ocrx.csv")

In [47]:
df_ocrx

,Unnamed: 0,Unnamed: 0.1,code_cis,denomination_bdpm,form,roa,status_amm,type_amm,commercialisation_state,date_amm,...,titulaire,surveillance,designation,substance_code,substance,dose,dose_ref,composant_type,num_linked,indications
0,0,0,61266250,"A 313 200 000 UI POUR CENT, pommade","[{'ansm_label': 'POMMADE', 'ocrx_label': 'POMM...","[{'ansm_label': 'CUTANÉE', 'ocrx_label': nan, ...",Autorisation active,Procédure nationale,Commercialisée,12/03/1998,...,PHARMA DEVELOPPEMENT,Non,pommade,77887,[{'ansm_label': 'CONCENTRAT DE VITAMINE A SYNT...,200 000 UI,100 g de pommade,SA,1.0,"['Dermatite de contact', ' Eczéma de contact']"
1,1,1,61266250,"A 313 200 000 UI POUR CENT, pommade","[{'ansm_label': 'POMMADE', 'ocrx_label': 'POMM...","[{'ansm_label': 'CUTANÉE', 'ocrx_label': nan, ...",Autorisation active,Procédure nationale,Commercialisée,12/03/1998,...,PHARMA DEVELOPPEMENT,Non,pommade,77887,[{'ansm_label': 'CONCENTRAT DE VITAMINE A SYNT...,200 000 UI,100 g de pommade,SA,1.0,"['Eczéma de contact', ' Dermatite de contact']"
2,2,2,66513085,"ABASAGLAR 100 unités/ml, solution injectable e...","[{'ansm_label': 'SOLUTION INJECTABLE', 'ocrx_l...","[{'ansm_label': 'SOUS-CUTANÉE', 'ocrx_label': ...",Autorisation active,Procédure centralisée,Commercialisée,09/09/2014,...,ELI LILLY NEDERLAND BV (PAY-BAS),Non,solution,44461,"[{'ansm_label': 'INSULINE GLARGINE', 'ocrx_lab...",100 unités,1 ml,SA,1.0,['Diabète']
3,3,3,64332894,"ABASAGLAR 100 unités/ml, solution injectable e...","[{'ansm_label': 'SOLUTION INJECTABLE', 'ocrx_l...","[{'ansm_label': 'SOUS-CUTANÉE', 'ocrx_label': ...",Autorisation active,Procédure centralisée,Commercialisée,09/09/2014,...,ELI LILLY NEDERLAND BV (PAY-BAS),Non,solution,44461,"[{'ansm_label': 'INSULINE GLARGINE', 'ocrx_lab...",100 unités,1 ml,SA,1.0,['Diabète']
4,4,4,66207341,"ABELCET 5 mg/ml, suspension à diluer pour perf...",[{'ansm_label': 'SUSPENSION À DILUER POUR PERF...,"[{'ansm_label': 'INTRAVEINEUSE', 'ocrx_label':...",Autorisation active,Procédure nationale,Commercialisée,10/06/1997,...,ACINO FRANCE,Non,suspension,1150,"[{'ansm_label': 'AMPHOTÉRICINE B', 'ocrx_label...",5 mg,1 mL de suspension,SA,1.0,['Aspergillose']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23880,23880,23880,68664125,"ZYPREXA VELOTAB 5 mg, comprimé orodispersible","[{'ansm_label': 'COMPRIMÉ ORODISPERSIBLE', 'oc...","[{'ansm_label': 'ORALE', 'ocrx_label': 'ORALE'...",Autorisation active,Procédure centralisée,Commercialisée,03/02/2000,...,ELI LILLY (PAYS BAS),Non,comprimé,17723,"[{'ansm_label': 'OLANZAPINE', 'ocrx_label': 'O...",5 mg,un comprimé,SA,1.0,['Schizophrénie']
23881,23881,23881,65016707,"ZYRTEC 10 mg/ml, solution buvable en gouttes","[{'ansm_label': 'SOLUTION BUVABLE EN GOUTTES',...","[{'ansm_label': 'ORALE', 'ocrx_label': 'ORALE'...",Autorisation active,Procédure de reconnaissance mutuelle,Commercialisée,14/11/1996,...,UCB PHARMA,Non,solution,60929,[{'ansm_label': 'DICHLORHYDRATE DE CÉTIRIZINE'...,"1,000 g",100 ml,SA,1.0,['Rhinite allergique']
23882,23882,23882,65016707,"ZYRTEC 10 mg/ml, solution buvable en gouttes","[{'ansm_label': 'SOLUTION BUVABLE EN GOUTTES',...","[{'ansm_label': 'ORALE', 'ocrx_label': 'ORALE'...",Autorisation active,Procédure de reconnaissance mutuelle,Commercialisée,14/11/1996,...,UCB PHARMA,Non,solution,60929,[{'ansm_label': 'DICHLORHYDRATE DE CÉTIRIZINE'...,"1,000 g",100 ml,SA,1.0,['Urticaire']
23883,23883,23883,64949486,"ZYRTECSET 10 mg, comprimé pelliculé sécable","[{'ansm_label': 'COMPRIMÉ PELLICULÉ SÉCABLE', ...","[{'ansm_label': 'ORALE', 'ocrx_label': 'ORALE'...",Autorisation active,Procédure nationale,Commercialisée,28/10/1994,...,UCB PHARMA,Non,comprimé,60929,[{'ansm_label': 'DICHLORHYDRATE DE CÉTIRIZINE'...,10 mg,un comprimé,SA,1.0,['Rhinite allergique']


In [48]:
df_ocrx_with_ade = pd.merge(df_ocrx,df_rcp,on="code_cis",how="left")

In [49]:
df_ocrx_with_ade['adverse_events_text'].isna().sum()

2627

In [50]:
# 2627 / 23885 have no adverse events

In [51]:
df_ocrx_with_ade.to_csv("df_ocrx_with_ade.csv")

In [52]:
df_ocrx_with_ade['form'].iloc[0]

"[{'ansm_label': 'POMMADE', 'ocrx_label': 'POMMADE', 'ansm_code': '298', 'ocrx_code': 'http://www.ocrx.ca/OCRx/4000000083', 'ansm_match_type': 'EXACT', 'ocrx_match_type': 'EXACT'}]"

In [53]:
df_ocrx_with_ade['substance'].iloc[6]

"[{'ansm_label': 'ARIPIPRAZOLE', 'ocrx_label': 'ARIPIPRAZOLE', 'ansm_code': '891', 'ocrx_code': 'http://www.ocrx.ca/OCRx/3000002188', 'ansm_match_type': 'EXACT', 'ocrx_match_type': 'EXACT'}]"

In [54]:
SNOMED CT pour matching
REST API pour match indications et ADEs



SyntaxError: invalid syntax (<ipython-input-54-e96558c5a44c>, line 1)